<a href="https://colab.research.google.com/github/Clarice-Satiko-Aoto/b2b-analyis/blob/feat/VendasB2B_feat_eng_metas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install workalendar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 60.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=a0db16dbca33ea5f3428778c5203032af107385e6143ab6ba0a9c6dcc3d4fb2c
  Stored in directory: /root/.cache/pip/wheels/d6/67/78/aa2e8d108639dd23a5e9e72a4fc88bb44f5541894382712f48
Successfully built pymeeus


# Bibliotecas

In [2]:
#importar bibliotecas
import pandas as pd

from workalendar.america import Brazil

# Crie um objeto de calendário brasileiro que exclui os feriados.
cal = Brazil()

#warnings
import warnings
warnings.filterwarnings("ignore")

# Carregar arquivos necessarios

In [24]:
#carregar arquivo

# df tratado
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/tratado.csv',
                 parse_dates=['data'])
#de para
dfDePara = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/de_para.csv')

#Meta 2022
meta22 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/vendas/data/Meta_2022.csv')

#Meta 2023
# mapeando o caminho
arquivo_excel = pd.ExcelFile('/content/drive/MyDrive/Colab Notebooks/vendas/data/Meta_2023.xlsx')
#lendo todas as tabelas
planilhas = arquivo_excel.parse(sheet_name=None,
                                header=2,
                                usecols=['Categoria Produto', 'Previsão Vendas', '1', '2', '3','4'])
#armazenando as planilhas
victor1_2023o = planilhas['1 - Victor Castro']
gabriel2_2023o = planilhas['2 - Gabriel Azevedo']

# Funções

In [4]:
def mudar_tipo_de_dados(dataframe, tipos_de_dados):
    """
    Muda o tipo de dados de colunas em um DataFrame.

    Parâmetros:
    - dataframe: DataFrame Pandas
    - tipos_de_dados: Lista de pares (nome da coluna, tipo de dados)
    """

    for coluna, tipo in tipos_de_dados:
        if coluna in dataframe.columns:
            dataframe[coluna] = dataframe[coluna].astype(tipo)

In [5]:
def dias_uteis_sem_feriado(df):
  """  dias uteis sem feriado no mes  """
  data_inicial = df.index.min()
  data_final = df.index.max()

  # Gere um índice de datas diárias.
  datas_diarias = pd.date_range(start=data_inicial, end=data_final, freq='D')

  # Filtrar as datas úteis, excluindo fins de semana e feriados brasileiros.
  dias_sem_feriado = [data for data in datas_diarias if data.weekday() < 5 and data not in cal.holidays()]
  return dias_sem_feriado

In [6]:
# Função para calcular a quantidade de dias úteis em um mês específico.

def calcular_dias_uteis_no_mes(row):
    """    calcular os dias uteis no mes    """
    ano, mes = row['data'].year, row['data'].month
    primeiro_dia = pd.Timestamp(year=ano, month=mes, day=1)
    ultimo_dia = pd.Timestamp(year=ano, month=mes, day=pd.Timestamp(year=ano, month=mes, day=1).days_in_month)

    datas_no_mes = pd.date_range(start=primeiro_dia, end=ultimo_dia)
    datas_uteis = [data for data in datas_no_mes if data.weekday() < 5]
    feriados = cal.holidays()
    datas_uteis = [data for data in datas_uteis if data not in feriados]
    return len(datas_uteis)

In [7]:
def transformar_meta_mensal_para_diaria(df):
  """Separando os dados de acordo com o nível gerente/categoria_id para que possa preencher
  com os valores da meta mensal no nível dias uteis"""

  meta_ano_gerente = []
  for ger in df['gerente_id'].unique():
    data = df[df['gerente_id']== ger]
    for cat in data['categoria_id'].unique():
      resample_ger_cat = data[data['categoria_id']==cat]
      df_diario = resample_ger_cat.reindex(dias_uteis_sem_feriado(resample_ger_cat))
      df_diario = df_diario.fillna(method='ffill')
      meta_ano_gerente.append(df_diario)
  # Concatene todos os DataFrames da lista em um único DataFrame.
  meta_concat = pd.concat(meta_ano_gerente)
  return meta_concat

In [8]:
#função para tratar os dados de 2023
#Padronizando a data
trimestre = {
    '1T': '2023-01-01',
    '2T': '2023-04-01',
    '3T': '2023-07-01',
    '4T': '2023-10-01'
}

def tratamento_dados_2023(df, id):
  #Adicionando a coluna com o gerente_id
  df['gerente_id'] = id

  #Extraindo apenas o id da categoria
  df['categoria_id'] = df['Categoria Produto'].str.extract('(\d+)')
  df.drop('Categoria Produto', axis=1, inplace=True)

  #calculando a meta trimestral
  df['1T'] = df['Previsão Vendas'] * df['1']
  df['2T'] = df['Previsão Vendas'] * df['2']
  df['3T'] = df['Previsão Vendas'] * df['3']
  df['4T'] = df['Previsão Vendas'] * df['4']

  #dropando as colunas que eu nao vou mais utilizar
  df.drop(columns=['Previsão Vendas', '1', '2', '3', '4'], axis=1, inplace=True)

  #utilizando a função metl para pivotar o dataframe
  df = pd.melt(df, id_vars=['gerente_id', 'categoria_id'], var_name='Trimestre', value_name='Valor')
  df.replace(trimestre, inplace=True)
  df['Trimestre'] = pd.to_datetime(df['Trimestre'])
  df.rename(columns={'Trimestre': 'data'}, inplace=True)
  # df.set_index('data', inplace=True)

  return df

In [9]:
def calcular_meta_diaria(df, col_meta):

  # Calcule os dias úteis para cada mês e adicione como uma nova coluna 'dias_uteis'.
  df['dias_uteis'] = df.apply(calcular_dias_uteis_no_mes, axis=1)

  # Distribua o valor de "$ Meta" igualmente pelos dias úteis.
  df['$_Meta_por_dia_util'] = round(df[col_meta] / df['dias_uteis'],2)
  df.reset_index(inplace=True)

  # filtrando as colunas que vamos utilizar e as datas do ano de 2022.
  meta_consolidado = df[['data','gerente_id', 'categoria_id', col_meta, '$_Meta_por_dia_util']]  #'dias_uteis',
  filtro_data = meta_consolidado['data'] >= '2022-01-01'
  meta_consolidado = meta_consolidado[filtro_data]
  return meta_consolidado


In [10]:
def calcular_meta_individual(df):
  """juntar a quantidade de funcionarios no datafrme
  das metas para calcular a meta diaria individual
  """
  meta_com_qde_func = df.merge(qde_func_por_gerente, on='gerente_id')
  meta_com_qde_func['meta_diaria_ind'] = round(meta_com_qde_func['$_Meta_por_dia_util'] / meta_com_qde_func['qde_funcionarios'],2)
  return meta_com_qde_func

In [11]:
def distribuir_meta_trimestral(df):
  """Transformar os dados mensais na granularidade diaria."""

  meta_tri = []
  for cat in df_metas_2023['categoria_id'].unique():
    filtro_cat = (df_metas_2023[df_metas_2023['categoria_id'] == cat])
    for ger in filtro_cat['gerente_id'].unique():
      filtro_ger = filtro_cat[filtro_cat['gerente_id']==ger]
      resample_cat_ger = filtro_ger.set_index('data').resample('D').ffill()
      df_diario = resample_cat_ger.reindex(dias_uteis_sem_feriado(resample_cat_ger))
      meta_tri.append(df_diario)
    #concatenando os dataframes
    meta_ger_23 = pd.concat(meta_tri)
    return meta_ger_23

# Dados auxiliares

## Mudar tipo de dado

In [12]:
tipos_de_dados = [('gerente_id', object), ('categoria_id', object), ('supervisor_id', object), ('funcionario_id', object)]

## Qde funcionarios por gerente

In [13]:
#Calculando q quantidade de funcionarios por gerente
qde_func_por_gerente = dfDePara['gerente_id'].value_counts().to_frame().reset_index().rename(columns={'gerente_id':'qde_funcionarios', 'index': 'gerente_id'})
qde_func_por_gerente

,gerente_id,qde_funcionarios
0,1,6
1,2,6


# Metas

## Meta 2022

In [25]:
meta22.head()

,Mês,gerente_id,categoria_id,$ Meta
0,Jan-22,1,1,22586
1,Jan-22,1,2,31244
2,Jan-22,1,3,16823
3,Jan-22,1,4,15085
4,Jan-22,2,1,18223


In [26]:
#Padronizar a data
#Jan-22 deixei como dez/2021 para que possamos preencher as datas de janeiro com o método ffill
mes = {
    'Jan-22': '2021-12-31',
    'Fev-22': '2022-02-28',
    'Mar-22': '2022-03-31',
    'Abr-22': '2022-04-30',
    'Mai-22': '2022-05-31',
    'Jun-22': '2022-06-30',
    'Jul-22': '2022-07-31',
    'Ago-22': '2022-08-31',
    'Set-22': '2022-09-30',
    'Out-22': '2022-10-31',
    'Nov-22': '2022-11-30',
    'Dez-22': '2022-12-31',
}
meta22.replace(mes, inplace=True)
meta22['Mês'] = pd.to_datetime(meta22['Mês'])
meta22.rename(columns={'Mês': 'data',
                       '$ Meta': 'meta_mensal'}, inplace=True)
meta22.set_index('data', inplace=True)

In [27]:
#mudar o tipo de dados
mudar_tipo_de_dados(meta22, tipos_de_dados)

In [28]:
#Transformar meta mensal para diária
meta22_concat = transformar_meta_mensal_para_diaria(meta22)

In [29]:
meta22_concat.reset_index(inplace=True)

In [30]:
#Calcular a meta diaria(dias uteis)
meta22_consolidado = calcular_meta_diaria(meta22_concat, 'meta_mensal')
meta22_consolidado.head()

,data,gerente_id,categoria_id,meta_mensal,$_Meta_por_dia_util
1,2022-01-03,1,1,22586.0,1075.52
2,2022-01-04,1,1,22586.0,1075.52
3,2022-01-05,1,1,22586.0,1075.52
4,2022-01-06,1,1,22586.0,1075.52
5,2022-01-07,1,1,22586.0,1075.52


In [31]:
#Calcular meta individual diária
meta22_com_qde_func = calcular_meta_individual(meta22_consolidado)
meta22_com_qde_func.head()

,data,gerente_id,categoria_id,meta_mensal,$_Meta_por_dia_util,qde_funcionarios,meta_diaria_ind
0,2022-01-03,1,1,22586.0,1075.52,6,179.25
1,2022-01-04,1,1,22586.0,1075.52,6,179.25
2,2022-01-05,1,1,22586.0,1075.52,6,179.25
3,2022-01-06,1,1,22586.0,1075.52,6,179.25
4,2022-01-07,1,1,22586.0,1075.52,6,179.25


In [32]:
#Merge com o de para para termos as informaçoes do supervidor_id e vendedor_id
meta22_compilado = meta22_com_qde_func.merge(dfDePara, on='gerente_id')
meta22_compilado.head()

,data,gerente_id,categoria_id,meta_mensal,$_Meta_por_dia_util,qde_funcionarios,meta_diaria_ind,supervisor_id,vendedor_id
0,2022-01-03,1,1,22586.0,1075.52,6,179.25,1,12
1,2022-01-03,1,1,22586.0,1075.52,6,179.25,2,1
2,2022-01-03,1,1,22586.0,1075.52,6,179.25,2,4
3,2022-01-03,1,1,22586.0,1075.52,6,179.25,2,11
4,2022-01-03,1,1,22586.0,1075.52,6,179.25,3,6


## Meta 2023

### Gerente Victor

In [33]:
victor1_2023_1  = victor1_2023o.copy()

In [34]:
victor1_2023_1

,Categoria Produto,Previsão Vendas,1,2,3,4
0,1 - Manga Longa,300000,0.10,0.40,0.33,0.17
1,2 - Manga Curta,950000,0.20,0.20,0.20,0.40
2,3 - Blazer,250000,0.12,0.33,0.35,0.20
3,4 - Camiseta,350000,0.20,0.20,0.20,0.40


In [35]:
#Transformar os dados de 2023
df_victor0 = tratamento_dados_2023(victor1_2023_1,1)


In [36]:
df_victor = df_victor0.copy()
df_victor.head()

,gerente_id,categoria_id,data,Valor
0,1,1,2023-01-01,30000.0
1,1,2,2023-01-01,190000.0
2,1,3,2023-01-01,30000.0
3,1,4,2023-01-01,70000.0
4,1,1,2023-04-01,120000.0


In [37]:
#Como eu nao consegui preencher o dataframe com os dados de novembro e dezembro,
# resolvi fazer na mão mesmo, criando o dataframe e concatenando com o dataframe original

faltante = [
    ['2023-11-30', 1,'1',51000.0],
    ['2023-12-31', 1,'1',51000.0],
    ['2023-11-30', 1,'2',380000.0],
    ['2023-12-31', 1,'2',380000.0],
    ['2023-11-30', 1,'3',50000.0],
    ['2023-12-31', 1,'3',50000.0],
    ['2023-11-30', 1,'4',140000.0],
    ['2023-12-31', 1,'4',140000.0],
            ]
df_faltante = pd.DataFrame(faltante, columns=['data', 'gerente_id', 'categoria_id', 'Valor'])
# Transformando as colunas 'data' e 'categoria_id' em tipo datetime e object.
df_faltante['data'] = pd.to_datetime(df_faltante['data'])

#concatenando os dois dataframes
df_victor_c = pd.concat([df_victor, df_faltante], ignore_index=True)
df_victor_c.tail()

,gerente_id,categoria_id,data,Valor
19,1,2,2023-12-31,380000.0
20,1,3,2023-11-30,50000.0
21,1,3,2023-12-31,50000.0
22,1,4,2023-11-30,140000.0
23,1,4,2023-12-31,140000.0


In [38]:
#mudar o tipo de dados
mudar_tipo_de_dados(df_victor_c, tipos_de_dados)

In [39]:
df_victor_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   gerente_id    24 non-null     object        
 1   categoria_id  24 non-null     object        
 2   data          24 non-null     datetime64[ns]
 3   Valor         24 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 896.0+ bytes


### Gerente Gabriel

In [40]:
gabriel2_2023_2 = gabriel2_2023o.copy()

In [41]:
df_gabriel = tratamento_dados_2023(gabriel2_2023_2,2)
df_gabriel.head()

,gerente_id,categoria_id,data,Valor
0,2,1,2023-01-01,42500.0
1,2,2,2023-01-01,150000.0
2,2,3,2023-01-01,39000.0
3,2,4,2023-01-01,40000.0
4,2,1,2023-04-01,170000.0


In [42]:
#concatenando com os dados faltantes

#Como eu nao consegui preencher o dataframe com os dados de novembro e dezembro,
# resolvi fazer na mão mesmo, criando o dataframe e concatenando com o dataframe original

faltante = [
    ['2023-11-30', 2,'1',85000.0],
    ['2023-12-31', 2,'1',85000.0],
    ['2023-11-30', 2,'2',262500.0],
    ['2023-12-31', 2,'2',262500.0],
    ['2023-11-30', 2,'3',65000.0],
    ['2023-12-31', 2,'3',65000.0],
    ['2023-11-30', 2,'4',70000.0],
    ['2023-12-31', 2,'4',70000.0],
            ]
df_faltante = pd.DataFrame(faltante, columns=['data', 'gerente_id', 'categoria_id', 'Valor'])
# Transformando as colunas 'data' e 'categoria_id' em tipo datetime e object.
df_faltante['data'] = pd.to_datetime(df_faltante['data'])

#concatenando os dois dataframes
df_gabriel_c = pd.concat([df_gabriel, df_faltante], ignore_index=True)
df_gabriel_c.tail()

,gerente_id,categoria_id,data,Valor
19,2,2,2023-12-31,262500.0
20,2,3,2023-11-30,65000.0
21,2,3,2023-12-31,65000.0
22,2,4,2023-11-30,70000.0
23,2,4,2023-12-31,70000.0


In [43]:
#mudar o tipo de dados
mudar_tipo_de_dados(df_gabriel_c, tipos_de_dados)


### Concatenar os dados dos dois gerentes

In [74]:
#concatenando os dados dos dois gerentes
df_metas_2023 = pd.concat([df_victor_c,df_gabriel_c ], ignore_index=True)
#adicionando a coluna trimestre para poder calcular os dias uteis
df_metas_2023['trimestre'] = df_metas_2023.data.dt.quarter
df_metas_2023.head()

,gerente_id,categoria_id,data,Valor,trimestre
0,1,1,2023-01-01,30000.0,1
1,1,2,2023-01-01,190000.0,1
2,1,3,2023-01-01,30000.0,1
3,1,4,2023-01-01,70000.0,1
4,1,1,2023-04-01,120000.0,2


In [75]:
#calcular os dias uteis no trimestre

feriados = cal.holidays()
df_resultado = pd.DataFrame()

for cat in df_metas_2023.categoria_id.unique():
  df_categoria = df_metas_2023.query("categoria_id == @cat")
  for ger in df_categoria.gerente_id.unique():
    df_gerente = df_categoria.query("gerente_id ==@ger")
    df_resampled = df_gerente.set_index('data').resample('D').ffill().reset_index()
    for tri in df_resampled.trimestre.unique():
      df_trimestre = df_resampled.query("trimestre == @tri")
      data_inicial = df_trimestre['data'].min()
      data_final = df_trimestre['data'].max()
      datas_trimestre = pd.date_range(start=data_inicial, end=data_final)
      datas_uteis = [data for data in datas_trimestre if data.weekday() < 5 ]
      datas_uteis = [data for data in datas_uteis if data not in feriados]
      quantidade_dias_uteis = len(datas_uteis)
      df_trimestre['dias_tri'] = quantidade_dias_uteis
      df_resultado = pd.concat([df_resultado, df_trimestre], ignore_index=True)

In [76]:
#Calcular os dias uteis no mes
df_resultado['dias_uteis_mes'] = df_resultado.apply(calcular_dias_uteis_no_mes, axis=1)
df_resultado.head()

,data,gerente_id,categoria_id,Valor,trimestre,dias_tri,dias_uteis_mes
0,2023-01-01,1,1,30000.0,1,65,22
1,2023-01-02,1,1,30000.0,1,65,22
2,2023-01-03,1,1,30000.0,1,65,22
3,2023-01-04,1,1,30000.0,1,65,22
4,2023-01-05,1,1,30000.0,1,65,22


In [77]:
#Calcular o valor da meta mensal
df_resultado['meta_mensal'] = df_resultado['Valor'] / df_resultado['dias_tri']* df_resultado['dias_uteis_mes']

In [80]:
#Verificar a saída do 1T gerente Victor
df_resultado.query("categoria_id =='4' & trimestre ==1").set_index('data').resample('M')['meta_mensal'].first().sum()

70000.0

In [66]:
#calcular o valor da meta diária
df_resultado['$_Meta_por_dia_util'] = df_resultado['meta_mensal']/df_resultado['dias_uteis_mes']


In [67]:
#Filtrar as colunas que vou precisar
df_resultado = df_resultado[['data', 'gerente_id', 'categoria_id', 'meta_mensal', '$_Meta_por_dia_util']]
df_resultado.head()

,data,gerente_id,categoria_id,meta_mensal,$_Meta_por_dia_util
0,2023-01-01,1,1,10153.846154,461.538462
1,2023-01-02,1,1,10153.846154,461.538462
2,2023-01-03,1,1,10153.846154,461.538462
3,2023-01-04,1,1,10153.846154,461.538462
4,2023-01-05,1,1,10153.846154,461.538462


In [68]:
#Calcular o valor da meta diaria individual
meta23_com_qde_func = calcular_meta_individual(df_resultado)
meta23_com_qde_func.head()

,data,gerente_id,categoria_id,meta_mensal,$_Meta_por_dia_util,qde_funcionarios,meta_diaria_ind
0,2023-01-01,1,1,10153.846154,461.538462,6,76.92
1,2023-01-02,1,1,10153.846154,461.538462,6,76.92
2,2023-01-03,1,1,10153.846154,461.538462,6,76.92
3,2023-01-04,1,1,10153.846154,461.538462,6,76.92
4,2023-01-05,1,1,10153.846154,461.538462,6,76.92


In [69]:
#Filtrar colunas necessárias
df_resultado22 = meta22_com_qde_func[['data', 'gerente_id', 'categoria_id', 'meta_mensal', '$_Meta_por_dia_util', 'meta_diaria_ind']]
df_resultado23 = meta23_com_qde_func[['data', 'gerente_id', 'categoria_id', 'meta_mensal', '$_Meta_por_dia_util', 'meta_diaria_ind']]

In [70]:
#Mudar o tipo dos dados
mudar_tipo_de_dados(df_resultado22,tipos_de_dados)

In [71]:
#Concatenar as metas 2022 e 2023
metas_compiladas = pd.concat([df_resultado22, df_resultado23])

In [72]:
#Salvar o arquivo compilado da meta
metas_compiladas.to_csv("/content/drive/MyDrive/Colab Notebooks/vendas/data/tratado/metas22-23.csv", index=False)